# 01 Import libraries

In [1]:
import numpy as np
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Using device: cpu


In [2]:
import pandas as pd

In [3]:
import importlib

In [4]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import lib
importlib.reload(lib)

In [6]:
from lib.lsp.basis_generator import bivariate_legendre, bivariate_chebyshev
from lib.lsp.learn_solution_path import adaptive_lsp, learn_solution_path
from lib.lsp.loss_fn_lsp import allocation_cost_no_con
from lib.lsp.utils_lsp import get_sup_error_lsp_2d
from lib.fast_tensor_data_loader import FastTensorDataLoader

# 02 Load data

In [7]:
# read file
decomp_cov_df = pd.read_csv('decomp_cov.csv')
mean_df = pd.read_csv('mean.csv')

In [8]:
decomp_cov = np.array(decomp_cov_df)
mean = np.array(mean_df).squeeze()

In [9]:
decomp_cov = torch.tensor(decomp_cov, dtype=torch.float32)
mean = torch.tensor(mean, dtype=torch.float32)

In [10]:
# full gradient descent uses all data points
GD_data_loader = FastTensorDataLoader(decomp_cov, mean, batch_size=len(decomp_cov), shuffle=False, )
# test data
test_data_loader = FastTensorDataLoader(decomp_cov, mean, batch_size=len(decomp_cov), shuffle=False, )

In [11]:
lam_max_2d = [1, 1]
lam_min_2d = [.2, 0]
input_dim = decomp_cov.shape[1]

In [12]:
# Read the CSV file into a DataFrame
truth = pd.read_csv('exact_soln_list_cov_001_100_100_no_con.csv')

# Display the DataFrame
truth

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.013926,-0.013637,-0.013352,-0.013070,-0.012791,-0.012515,-0.012242,-0.011973,-0.011707,-0.011445,...,-0.000112,-0.000088,-0.000068,-0.000050,-0.000034,-0.000022,-0.000012,-0.000006,-0.000001,0.0
1,-0.014002,-0.013711,-0.013424,-0.013140,-0.012859,-0.012582,-0.012308,-0.012038,-0.011770,-0.011506,...,-0.000112,-0.000089,-0.000068,-0.000050,-0.000035,-0.000022,-0.000012,-0.000006,-0.000001,0.0
2,-0.014078,-0.013786,-0.013497,-0.013211,-0.012929,-0.012650,-0.012375,-0.012103,-0.011834,-0.011568,...,-0.000113,-0.000089,-0.000068,-0.000050,-0.000035,-0.000022,-0.000013,-0.000006,-0.000001,0.0
3,-0.014155,-0.013861,-0.013571,-0.013284,-0.013000,-0.012719,-0.012442,-0.012169,-0.011898,-0.011631,...,-0.000113,-0.000090,-0.000069,-0.000050,-0.000035,-0.000022,-0.000013,-0.000006,-0.000001,0.0
4,-0.014234,-0.013938,-0.013646,-0.013357,-0.013071,-0.012789,-0.012511,-0.012235,-0.011963,-0.011695,...,-0.000114,-0.000090,-0.000069,-0.000051,-0.000035,-0.000023,-0.000013,-0.000006,-0.000001,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.034586,-0.033755,-0.032940,-0.032141,-0.031358,-0.030589,-0.029835,-0.029096,-0.028371,-0.027660,...,-0.000249,-0.000197,-0.000151,-0.000111,-0.000077,-0.000049,-0.000028,-0.000012,-0.000003,0.0
96,-0.035320,-0.034465,-0.033626,-0.032804,-0.031998,-0.031208,-0.030434,-0.029675,-0.028930,-0.028200,...,-0.000253,-0.000200,-0.000153,-0.000112,-0.000078,-0.000050,-0.000028,-0.000012,-0.000003,0.0
97,-0.036098,-0.035216,-0.034352,-0.033505,-0.032676,-0.031863,-0.031066,-0.030285,-0.029520,-0.028770,...,-0.000257,-0.000203,-0.000155,-0.000114,-0.000079,-0.000051,-0.000029,-0.000013,-0.000003,0.0
98,-0.036925,-0.036015,-0.035123,-0.034249,-0.033394,-0.032556,-0.031735,-0.030931,-0.030143,-0.029372,...,-0.000261,-0.000206,-0.000158,-0.000116,-0.000080,-0.000051,-0.000029,-0.000013,-0.000003,0.0


In [13]:
true_losses = truth.to_numpy()

In [14]:
loss_fn = allocation_cost_no_con

# 03 Adaptively learn solution path

In [15]:
max_epochs = 10

In [16]:
phi_lam = bivariate_legendre
# phi_lam = bivariate_chebyshev

In [17]:
basis_list = [4, 9, 16, 25, 25]
lr_list = [.1, .1, .1, .1, .1]

In [24]:
def thresh_basis(basis_dim):
    return 0.1**(basis_dim)

In [38]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, breaks, break_grad_list = adaptive_lsp(input_dim, 4, 25,
                                        phi_lam, max_epochs, GD_data_loader,  test_data_loader,
                                        loss_fn, lam_min_2d, lam_max_2d, true_losses, init_lr=.1,
                                        diminish=False, weighted_avg=False,
                                        intercept=False, thresh_basis=thresh_basis, record_frequency=1,
                                        record_fctn=get_sup_error_lsp_2d,
                                        opt_method='lbfgs', distribution='uniform', device=device, 
                                        basis_list=basis_list, lr_list=lr_list,
                                        trace_frequency=1)

********** now running lsp with #basis dimension = 4 ***********
lr = 0.1
--------approximate solution path for # itr = 20 complete--------
# epoch: 1	 sup error: 0.005362487037660797
--------approximate solution path for # itr = 40 complete--------
# epoch: 2	 sup error: 0.003965313185932358
--------approximate solution path for # itr = 60 complete--------
# epoch: 3	 sup error: 0.0030637184278986054
--------approximate solution path for # itr = 80 complete--------
# epoch: 4	 sup error: 0.002837526251556595
--------approximate solution path for # itr = 88 complete--------
# epoch: 5	 sup error: 0.0028241524593851167
********** now running lsp with #basis dimension = 9 ***********
lr = 0.1
--------approximate solution path for # itr = 20 complete--------
# epoch: 1	 sup error: 0.0019342551128885346
--------approximate solution path for # itr = 40 complete--------
# epoch: 2	 sup error: 0.0015728274481317597
--------approximate solution path for # itr = 60 complete--------
# epoch: 3	 

In [46]:
pd.DataFrame(np.column_stack((break_grad_list, breaks)), columns=['break_grads', 'break_func_evals']).to_csv('LSP_results_exact_boosted_breaks_2d_no_con.csv', index=False)

In [44]:
last_grad_call = break_grad_list[len(break_grad_list)-1]

In [47]:
break_grad_list

[5, 15, 25, 35]

In [43]:
break_grad_list = [5, 15, 25, 35]

In [45]:
file_path = 'LSP_results_exact_boosted_2d_no_con.csv'

LSP_results_exact = pd.DataFrame(np.column_stack((np.arange(1, last_grad_call+1), num_itr_history, sup_err_history)), columns=['num_grad_call', 'num_itr', 'sup_err'])

# Save the DataFrame to a CSV file
LSP_results_exact.to_csv(file_path, index=False)

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df

,num_grad_call,num_itr,sup_err
0,1.0,20.0,0.005362
1,2.0,40.0,0.003965
2,3.0,60.0,0.003064
3,4.0,80.0,0.002838
4,5.0,88.0,0.002824
5,6.0,108.0,0.001934
6,7.0,128.0,0.001573
7,8.0,148.0,0.000877
8,9.0,168.0,0.000518
9,10.0,188.0,0.000517
